In [30]:
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from base import BaseDataLoader
import pandas as pd
import gzip
import numpy as np
import re

In [32]:
def buildGlove():
    words = []
    idx = 0
    word2idx = {}
    # vectors = bcolz.carray(np.zeros(1), rootdir=f'./data/glove_6B_50.dat', mode='w')
    vectors = []
    with open(f'./data/glove.6B.50d.txt', 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            words.append(word)
            word2idx[word] = idx
            idx += 1
            vect = np.array(line[1:]).astype(np.float)
            vectors.append(vect)
    vectors = np.reshape(vectors, (400000, 50))
    glove = {w: vectors[word2idx[w]] for w in words}
    return  glove

In [109]:
class AmznDataLoader():
    """
    AMZN data loading demo using BaseDataLoader
    """
    def __init__(self):
        self.df = self.getDF('./data/reviews_Amazon_Instant_Video_5.json.gz')
#         display(self.df)
#         print(self.df.loc[0][['overall','reviewText']])
        self.df = self.df[['reviewText', 'overall']]
        self.df['reviewText'] = self.df['reviewText'].apply(lambda x : self.title_parsing(x))
        X = self.df['reviewText']
        y = self.df['overall']
        self.maxNumberWords = 0 
        for x in self.df['reviewText']:
            length = len(x)
            if length > self.maxNumberWords:
                self.maxNumberWords = length
        print(self.maxNumberWords)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
    def title_parsing(self, title):
        title = re.sub('[^a-zA-Z]', ' ', str(title))
        title = title.lower()
        title = title.split()
        return title
    def get_batches(self, X, y, batch_size, glove):
        batch_x = []
        batch_y = []
        choices = np.random.choice(X.shape[0], batch_size)
        for idx in choices:
            batch_x.append(X.iloc[idx])
            batch_y.append(y.iloc[idx])
            
            
        numberWords = len(batch_x)
        glove_tensor = np.zeros((batch_size, 50, self.maxNumberWords))
        words_found = 0
        for j, sentence in enumerate(batch_x):
            for i, word in enumerate(sentence):
                try: 
                    glove_tensor[j,:, i] = glove[word]
                    words_found += 1
                except KeyError:
                    glove_tensor[j,:,i] = np.random.normal(scale=0.6, size=(50, ))
                    
        batch_x = glove_tensor
        print(batch_x.shape)
        return batch_x, batch_y
    
    def parse(self, path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield eval(l)
        
    def getDF(self, path):
        i = 0
        df = {}
        for d in self.parse(path):
            df[i] = d
            i += 1
        return pd.DataFrame.from_dict(df, orient='index')

In [110]:
data_loader = AmznDataLoader()
glove = buildGlove()
a, b = data_loader.get_batches(data_loader.X_train, data_loader.y_train, batch_size=256, glove=glove)

3068
(256, 50, 3068)


In [87]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

In [137]:

class FirstModel(nn.Module):
    def __init__(self, num_classes=10):
        super(FirstModel, self).__init__()
        self.drop1 = nn.Dropout(p=0.5)
        self.conv1 = nn.Conv1d(in_channels=50, out_channels=200, kernel_size=4, padding=2)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=200, kernel_size=5, padding=2)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.drop2 = nn.Dropout(p=0.15)
        self.rnn = nn.GRU(input_size=1534, hidden_size=)

    def forward(self, x):
        x = self.drop1(x)
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x))
        x1 = self.maxpool(x1)
        x2 = self.maxpool(x2)
        print(x1.size(), x2.size())
        x = torch.cat((x1,x2), 1)
        print(x.size())
        x = self.drop2(x)
        x = pack_padded_sequence(x, lengths, batch_first=True)
        x, hidden =self.rnn(x)
        
        
        return x

In [138]:
import torch.optim as optim
model = FirstModel().float()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [139]:

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(2):
        # get the inputs
        inputs, labels = data_loader.get_batches(data_loader.X_train, data_loader.y_train, batch_size=256, glove=glove)
        inputs = torch.from_numpy(inputs).float()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0


(256, 50, 3068)
torch.Size([256, 200, 1534]) torch.Size([256, 200, 1534])
torch.Size([256, 400, 1534])


AttributeError: 'list' object has no attribute 'size'